In [1]:
import bhnerf
from astropy import units
import jax

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
from flax.training import checkpoints
from pathlib import Path
import ruamel.yaml as yaml
from matplotlib import animation

# Runing on 2 GPUs
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

# Import script function
import sys
sys.path.append('../scripts/')
from Fit_ALMA_LP_Apr11_SgrA_Flare import preprocess_data

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-0wj560kx because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


Welcome to eht-imaging! v 1.2.2 



2023-06-16 16:35:44.736284: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /.singularity.d/libs


In [2]:
def sample_3D_recovery(checkpoint_dir, coords, chunk=-1):
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
    state = checkpoints.restore_checkpoint(checkpoint_dir, None)
    emission = bhnerf.network.sample_3d_grid(predictor.apply, state['params'], coords=coords, chunk=chunk)
    return emission

def animate(movie, t_frames, fps=10, output=None, writer='ffmpeg'):

    # Image animation function (called sequentially)
    def animate_frame(i):
        #axes.set_title('t={:1.2f} UTC'.format(t_frames[i]))
        im.set_array(movie[i])
        return im
    
    num_frames = movie.shape[0]
    
    fig, axes = plt.subplots(1,1, figsize=(4,4))
    im =  axes.imshow(movie[0])
    #axes.set_title('t={:1.2f} UTC'.format(t_frames[0]), fontsize=16)
    axes.set_axis_off()
    
    plt.tight_layout()
    anim = animation.FuncAnimation(fig, animate_frame, frames=num_frames, interval=1e3 / fps)

    if output is not None:
        anim.save(output, writer=writer, fps=fps)
    return anim

In [3]:
recovery_path = Path('../checkpoints/alma/intrinsic_fits/vertical_b_variable_pixels1/')
basename = 'inc_{:.1f}.seed_{}'

outpath = Path(os.path.join(*recovery_path.parts[2:]))
outpath.mkdir(parents=True, exist_ok=True)

with open(recovery_path.joinpath('config.yml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.Loader)

locals().update(config['preprocess'])
locals().update(config['model'])
locals().update(config['optimization'])

# Preprocess / split data to train/validation
target, t_frames = preprocess_data(**config['preprocess'])
train_idx = t_frames <= config['preprocess']['t_start']*units.hr + train_split*units.min
val_idx = t_frames > config['preprocess']['t_start']*units.hr + train_split*units.min
data_train, data_val  = target[train_idx], target[val_idx] 
t_train, t_val = t_frames[train_idx], t_frames[val_idx]
rmax = fov_M / 2
if rmin == 'ISCO': rmin = float(bhnerf.constants.isco_pro(spin))

In [16]:
spin_loss = pd.read_csv(recovery_path.joinpath('spin_loss.csv'), index_col=0)

plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})

%matplotlib widget
plt.figure(figsize=(5,4))
plt.errorbar(spin_grid, np.nanmean(np.log10(spin_loss), axis=1), np.nanstd(np.log10(spin_loss), axis=1), color='tab:orange', marker='^', mfc='r', mec='r', label='data-fit', markersize=5)

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.axhline(0, color='black', linestyle='--',linewidth=0.75)
plt.title(r'Spin data-fit: $\log \chi^2(a | {\bf w}^\star)$', fontsize=16)
# plt.legend(loc='best', fontsize=14)
plt.savefig(outpath.joinpath('spin_loss.pdf'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
spin_loss = pd.read_csv(recovery_path.joinpath('spin_loss.csv'), index_col=0)
spin_loss_subrays = pd.read_csv(recovery_path.joinpath('spin_loss_subrays_10.csv'), index_col=0)

plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica"})

%matplotlib widget
plt.figure(figsize=(5,4))
plt.errorbar(spin_grid, np.nanmean(np.log10(spin_loss), axis=1), np.nanstd(np.log10(spin_loss), axis=1), color='tab:orange', marker='^', mfc='r', mec='r', label='data-fit', markersize=5)
plt.errorbar(spin_grid, np.nanmean(np.log10(spin_loss_subrays), axis=1), np.nanstd(np.log10(spin_loss_subrays), axis=1), color='tab:blue', marker='o', mfc='b', mec='b', label='validation', markersize=5)

plt.xticks(fontsize='14')
plt.yticks(fontsize='14')
plt.axhline(0, color='black', linestyle='--',linewidth=0.75)
plt.title(r'Spin data-fit: $\log \chi^2(a | {\bf w}^\star)$', fontsize=16)
plt.legend(loc='best', fontsize=14)
plt.savefig(outpath.joinpath('spin_loss_w_subray_validation.pdf'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
raytrace_args = image_plane_model(np.deg2rad(inclination), spin)
datafit, image_plane = image_plane_fit(raytrace_args, checkpoint_dir, t_train, data_train)

In [11]:
%matplotlib widget
plt.rcParams.update({"text.usetex": True, "font.family": "Helvetica",})
axes = bhnerf.visualization.plot_stokes_lc(data_train[:,1:], ['Q','U'], t_train, label='ALMA', color='black', plot_qu=True)
bhnerf.visualization.plot_stokes_lc(model[:,1:], ['Q','U'], t_train, axes=axes, color='r', fmt='x', label='Model', plot_qu=True)

titles = [r'$I_Q$ datafit', r'$I_U$ datafit', r'$Q-U$ datafit']
for ax, title in zip(axes, titles):
    ax.set_title(title, fontsize=16)
    ax.legend()
    
axes[0].set_xlabel('Time [UT]', fontsize=12)
axes[1].set_xlabel('Time [UT]', fontsize=12)
plt.tight_layout()

savepath = outpath.joinpath(basename.format(inclination, seed))
savepath.mkdir(parents=True, exist_ok=True)
plt.savefig(savepath.joinpath('QU.datafit.pdf'), bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [266]:
fov_rad = (fov_M * consts.GM_c2(consts.sgra_mass) / consts.sgra_distance.to('m')) * units.rad
psize = fov_rad.value / num_alpha

im = eh.image.Image(np.rot90(image_plane[0,0], 2), psize, 0, 0)
im.display();

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [44]:
%matplotlib widget
movie_list = [xr.DataArray(image_plane[:, i], dims=['t','beta','alpha']) for i in range(image_plane.shape[1])]
fig, axes = plt.subplots(1, 3, figsize=(10, 3))
bhnerf.visualization.animate_movies_synced(movie_list, axes, titles=['I', 'Q', 'U'], vmin=[0, -.02, -.02], vmax=[.03, .02, .01])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Rotate 3D volume
---
Animate an average recovery 3D volume

In [11]:
seeds = range(4)
inc = 12

zenith = np.deg2rad(60)
view_azimuths = np.linspace(0, 2*np.pi, 30)

jit = True
resolution = 256
bh_radius = 1 + np.sqrt(1-spin**2)
cam_r = 55.
linewidth = 0.14
norm_const =  0.05

visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)

images = []
for azimuth in tqdm(view_azimuths):
    emission = 0
    visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=azimuth, zenith=zenith)
    for  seed in seeds:
        checkpoint_dir = recovery_path.joinpath(basename.format(inc, seed))
        emission += sample_3D_recovery(checkpoint_dir, visualizer.coords, chunk=32) / len(seeds)

    image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, bh_radius=bh_radius, linewidth=linewidth).clip(a_max=1)
    images.append(image)
    
movie = np.array(images)
np.save('alma/intrinsic_fits/vertical_b_variable_pixels1/inc12.seeds0-4.recovery_3D_rotation_zenith_60.npy', movie)

  0%|          | 0/30 [00:00<?, ?it/s]

In [22]:
%matplotlib widget
animate(movie, t_frames, output='alma/intrinsic_fits/vertical_b_variable_pixels1/inc12.seeds0-4.recovery_3D_rotation_zenith_60.gif')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Temporal evolution

In [4]:
def sample_3D_recovery(checkpoint_dir, coords, t_frame=0, t_start_obs=0, Omega=0, chunk=-1):
    predictor = bhnerf.network.NeRF_Predictor.from_yml(checkpoint_dir)
    state = checkpoints.restore_checkpoint(checkpoint_dir, None)
    emission = bhnerf.network.sample_3d_grid(predictor.apply, state['params'], t_frame, t_start_obs, Omega, coords=coords, chunk=chunk)
    return emission

In [9]:
seeds = range(4)
inc = 12

zenith = np.deg2rad(30)
azimuth = 0

jit = True
resolution = 256
bh_radius = 1 + np.sqrt(1-spin**2)
cam_r = 55.
linewidth = 0.14
norm_const =  0.05

visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)
visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=azimuth, zenith=zenith)

# Keplerian velocity and Doppler boosting
rot_sign = {'cw': -1, 'ccw': 1}
r = np.sqrt(visualizer.coords[0]**2 + visualizer.coords[1]**2 + visualizer.coords[2]**2)
Omega = rot_sign[Omega_dir] * np.sqrt(1) / (r**(3/2) + spin * np.sqrt(1))
    
images = []
for t in tqdm(np.linspace(t_train[0], t_train[-1], 100)):
    emission = 0
    for seed in seeds:
        checkpoint_dir = recovery_path.joinpath(basename.format(inc, seed))
        emission += sample_3D_recovery(checkpoint_dir, visualizer.coords, t, t_start_obs, Omega, chunk=32) / len(seeds)
        
    image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, bh_radius=bh_radius, linewidth=linewidth).clip(a_max=1)
    images.append(image)

movie = np.array(images)
np.save('alma/intrinsic_fits/vertical_b_variable_pixels1/inc12.seeds0-4.recovery_3D_evolution_zenith_30.npy', movie)

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
%matplotlib widget
animate(movie, t_frames, output='alma/intrinsic_fits/vertical_b_variable_pixels1/inc12.seeds0-4.recovery_3D_evolution_zenith_30.gif')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
seeds = range(4)
inc = 12

zenith = np.deg2rad(30)
azimuth = 0

jit = False
resolution = 64
bh_radius = 1 + np.sqrt(1-spin**2)
cam_r = 55.
linewidth = 0.14
norm_const =  0.05

visualizer = bhnerf.visualization.VolumeVisualizer(resolution, resolution, resolution)
visualizer.set_view(cam_r=cam_r, domain_r=rmax, azimuth=azimuth, zenith=zenith)

# Keplerian velocity and Doppler boosting
rot_sign = {'cw': -1, 'ccw': 1}
r = np.sqrt(visualizer.coords[0]**2 + visualizer.coords[1]**2 + visualizer.coords[2]**2)
Omega = rot_sign[Omega_dir] * np.sqrt(1) / (r**(3/2) + spin * np.sqrt(1))
    
images = []
seed = 2
checkpoint_dir = recovery_path.joinpath(basename.format(inc, seed))
emission = sample_3D_recovery(checkpoint_dir, visualizer.coords, t_start_obs, t_start_obs , chunk=32) / len(seeds)

image = visualizer.render(emission / norm_const, facewidth=1.9*rmax, jit=jit, bh_radius=bh_radius, linewidth=linewidth).clip(a_max=1)


In [12]:
%matplotlib widget
plt.imshow(images[10]-images[0])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …